# Demo 

Code for demo

In [ ]:
import json
import urllib.request
import random
import pandas as pd

IPADDRESS = '20.98.127.24'
base_uri = f'http://{IPADDRESS}:8000'

dataset = pd.read_csv("https://azuremlexamples.blob.core.windows.net/datasets/iris.csv")
X=dataset[['sepal_length','sepal_width','petal_length','petal_width']]

def score(X, tenant):
    # tenant = random.choices(["tenant1","tenant2", "tenant3","tenant4","tenant5","tenant6","tenant7", "tenant8","tenant9"])[0]
    len = random.randint(2,50)
    data = {"tenant":tenant, "data": X.head(len).to_numpy().tolist()}
    body = str.encode(json.dumps(data))
    url = f'{base_uri}/score'
    headers = {'Content-Type':'application/json'}
    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)
        result = response.read()
        print(json.loads(result))
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))

def get_deploymentmap():
    headers = {'Content-Type':'application/json'}
    req = urllib.request.Request(url=f'http://{IPADDRESS}:8000/get_deploymentmap', headers=headers, method="GET")

    try:
        response = urllib.request.urlopen(req)
        result = response.read()
        print(json.loads(result))
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))

def get_dynamic_tenantmap():
    headers = {'Content-Type':'application/json'}
    req = urllib.request.Request(url=f'http://{IPADDRESS}:8000/get_dynamic_tenantmap', headers=headers, method="GET")

    try:
        response = urllib.request.urlopen(req)
        result = response.read()
        print(json.loads(result))
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))
        
def get_dedicated_tenantmap():
    headers = {'Content-Type':'application/json'}
    req = urllib.request.Request(url=f'http://{IPADDRESS}:8000/get_dedicated_tenantmap', headers=headers, method="GET")

    try:
        response = urllib.request.urlopen(req)
        result = response.read()
        print(json.loads(result))
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))


def updated_dedicated_pool(mapping):
    body = str.encode(json.dumps({"mapping":mapping}))
    url = f'{base_uri}/update_dedicated_pool'
    headers = {'Content-Type':'application/json'}
    req = urllib.request.Request(url, body, headers)
    try:
        response = urllib.request.urlopen(req)
        result = response.read()
        print(json.loads(result))
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))

[score(X, f"tenant{i}") for i in range(1, 9)]
mapping ={"tenant9":"deployment9","tenant10":"deployment10"}
updated_dedicated_pool(mapping)

## Scenario

100 models in total in AML 

Due to limisted resouces (CPU and Memory), we decided to load 10 models

Let's check which models are loaded in Hot memory

![](../../images/01_Deployment_RayServer.png)

In [9]:
# headers = {'Content-Type':'application/json'}
# req = urllib.request.Request(url=f'http://{IPADDRESS}:8000/get_deploymentmap', headers=headers, method="GET")

# response = urllib.request.urlopen(req)
# result = response.read()
# print(json.loads(result))
get_deploymentmap()

["Deployment1", "Deployment2", "Deployment3", "Deployment4", "Deployment5", "Deployment6", "Deployment7", "Deployment8", "Deployment9", "Deployment10", "Deploymentx"]


![](../../images/02_Deployment_Tenant__RayServer.png)

In [13]:
# http://{IPADDRESS}:8000/get_dynamic_tenantmap
get_dynamic_tenantmap()

{"tenant2": "deployment4", "tenant5": "deployment6", "tenant6": "deployment2", "tenant7": "deployment7", "tenant8": "deployment8", "tenant54": "deployment1", "tenant45": "deployment3", "tenant1": "deployment5"}


Test inference as `tenant1` user

In [12]:
%%time
score(X, "tenant1")

{'deployment': 'Deployment5', 'model': 'tenant1', 'prediction': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
CPU times: user 4.32 ms, sys: 0 ns, total: 4.32 ms
Wall time: 262 ms


Test inference as `tenant2` user

In [14]:
%%time
score(X, "tenant2")

{'deployment': 'Deployment4', 'model': 'tenant2', 'prediction': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
CPU times: user 1.9 ms, sys: 1.42 ms, total: 3.32 ms
Wall time: 260 ms


Test inference as `tenant45` user

In [16]:
%%time
score(X, "tenant39")

{'deployment': 'Deploymentx', 'model': 'tenant39', 'prediction': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
CPU times: user 6.51 ms, sys: 15.3 ms, total: 21.8 ms
Wall time: 2.35 s


Check models in memory (Hot)

![](../../images/03_Deployment_Tenant_NewTenant_RayServer.png)

In [17]:
# http://{IPADDRESS}:8000/get_dynamic_tenantmap
get_dynamic_tenantmap()

{"tenant2": "deployment4", "tenant6": "deployment2", "tenant7": "deployment7", "tenant8": "deployment8", "tenant54": "deployment1", "tenant45": "deployment3", "tenant1": "deployment5", "tenant39": "deployment6"}


And run inference again as the last `tenant45` user

In [18]:
%%time
score(X, "tenant39")

{'deployment': 'Deployment6', 'model': 'tenant39', 'prediction': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
CPU times: user 2.37 ms, sys: 1.76 ms, total: 4.14 ms
Wall time: 254 ms


In [19]:
# http://{IPADDRESS}:8000/get_dynamic_tenantmap
get_dynamic_tenantmap()

{"tenant2": "deployment4", "tenant6": "deployment2", "tenant7": "deployment7", "tenant8": "deployment8", "tenant54": "deployment1", "tenant45": "deployment3", "tenant1": "deployment5", "tenant39": "deployment6"}


Need to pin some of users's model regardless of the frequency of inference

API to update dedicated pool mapping. Try this to assign a tenant to a dedicated deployment

In [20]:
# http://{IPADDRESS}:8000/get_dedicated_tenantmap
get_dedicated_tenantmap()

{"tenant9": "deployment9", "tenant10": "deployment10"}


Test inference as exsiting tenant user in the dedicated memory (Hot)

In [21]:
%%time
score(X, "tenant9")

{'deployment': 'Deployment9', 'model': 'tenant9', 'prediction': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
CPU times: user 2.01 ms, sys: 1.48 ms, total: 3.49 ms
Wall time: 272 ms


![](../../images/04_Deployment_Tenant_DedicatedTenant_RayServer.png)

In [22]:
mapping ={"tenant91":"deployment9","tenant92":"deployment10"}
updated_dedicated_pool(mapping)

{'tenant91': 'deployment9', 'tenant92': 'deployment10'}


Test inference as `tenant91` or `tenant92`

In [23]:
%%time
score(X, "tenant91")

{'deployment': 'Deployment9', 'model': 'tenant91', 'prediction': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 258 ms


In [24]:
%%time
score(X, "tenant92")

{'deployment': 'Deployment10', 'model': 'tenant92', 'prediction': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
CPU times: user 0 ns, sys: 3.46 ms, total: 3.46 ms
Wall time: 257 ms


How about the old `tenant9`?

In [27]:
%%time
score(X, "tenant9")

{'deployment': 'Deployment2', 'model': 'tenant9', 'prediction': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
CPU times: user 4.59 ms, sys: 0 ns, total: 4.59 ms
Wall time: 253 ms


In [28]:
# http://{IPADDRESS}:8000/get_dynamic_tenantmap
get_dynamic_tenantmap()

{"tenant2": "deployment4", "tenant7": "deployment7", "tenant8": "deployment8", "tenant54": "deployment1", "tenant45": "deployment3", "tenant1": "deployment5", "tenant39": "deployment6", "tenant9": "deployment2"}


Check tenants in memory (Hot) again

In [29]:
# http://{IPADDRESS}:8000/get_dedicated_tenantmap
get_dedicated_tenantmap()

{"tenant91": "deployment9", "tenant92": "deployment10"}
